<a href="https://colab.research.google.com/github/hyd3nekosuki/RPDsummer2024/blob/main/%E7%82%89%E7%89%A9%E7%90%86%E5%A4%8F%E6%9C%9F%E3%82%BB%E3%83%9F%E3%83%8A%E3%83%BCPOD%E6%BC%94%E7%BF%92.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Google Colaboratoryを用いたPOD演習**


# **Google Colaboratoryの使い方**

Google Colaboratoryを用いることで、手持ちのPCに[Python](https://www.python.jp/pages/about.html)計算環境を整える必要なく、ウェブブラウザ上からPythonプログラムを容易に作成することができる。

Python言語そのものの文法など詳細については、各自インターネットで検索したり、別の参考文献を読んでほしい。例えば
 - ["Pythonプログラミング入門," 東京大学 数理・情報教育研究センター (2024).](https://utokyo-ipp.github.io/IPP_textbook.pdf)
 - [喜多 一, "プログラミング演習 Python 2019," (2020).](https://repository.kulib.kyoto-u.ac.jp/dspace/bitstream/2433/245698/1/Version2020_02_13_01.pdf)


### **pythonプログラムの実行練習**

まずは簡単な例として、以下のプログラムを実行してみよう。

\\

プログラムが記述されたセルを選択(マウスでクリック)し

①Shift+Enterを押すか

②マウスで左上部の[　] or ▶ ををクリックする

ことで作成したプログラムを実行することができる。

In [ ]:
a = 1.41421356

print(a) # 値を表示
print(id(a)) # 変数aのアドレスを表示

b=a
print(id(b)) # 変数bのアドレスを表示

### **繰り返し処理**

Pythonにおけるfor文の基本的な記述方法は以下のとおり。変数iが終了値を超えたら反復を終えるので注意。


for i in range(繰り返し回数):

  　～繰り返したい処理～


C言語と異なり、Pythonのfor文やif文では、同じ幅だけ**字下げ**することによって、処理の単位(ブロック)を表現している。同一のブロック内では文頭の字下げの数を同じにすること。

In [ ]:
keff = 0.9

neutrons = 100
for i in range(1, 11, 1):
    neutrons = keff*neutrons
    print(i, neutrons)


### **分岐処理**

if 条件式①:

  　～条件式①を満足する(True)の時に行う処理～

elif 条件式②:

  　～条件式②を満足する(True)の時に行う処理～

elif 条件式③:

  　～条件式③を満足する(True)の時に行う処理～

…

else:

  　～以上の条件をすべて満足しなかった時に行う処理～


In [ ]:
import math

nuSigF = 1.2
SigA = 0.5
D = 1.0
B = math.sqrt(0.7) # np.sqrt: 平方根
B2 = B*B

keff = nuSigF/(SigA + D*B2) # エネルギー1群、1領域均質体系の実効増倍率

print("keff = {0}".format(keff)) # {}の中にformatで指定した変数が出力される
if keff<1:
    print("subcritical: 未臨界")
elif keff>1:
    print("supercritical: 超臨界")
else:
    print("critical: 臨界")

# **前準備**

POD基底を作成するための演習を行うための、各種関数を定義するため
これ以降のpythonプログラムを全て実行してください。

(or ▶をクリック)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
from numba import njit

In [ ]:
def calcMatrixA(D, Dcor, SigA, dx, matid, albedo):
    NX  = len(s)
    A = sp.lil_matrix((NX, NX))

    for i in range(NX):
        D0  = D[i]
        dx0 = dx[i]
        if i==0:
            a = albedo[0]
            Dxm = -(1-a)*D0/(2*(1+a)*D0 + (1-a)*dx0/2)
        else:
            Dm  = D[i-1]
            dxm = dx[i-1]
            Dxm = -2*Dm*D0/(Dm*dx0 + D0*dxm)
            Axm = (Dxm-Dcor[i])/dx0
            A[i,i-1] = Axm
        if i==(NX-1):
            a = albedo[1]
            Dxp = -(1-a)*D0/(2*(1+a)*D0 + (1-a)*dx0/2)
        else:
            Dp  = D[i+1]
            dxp = dx[i+1]
            Dxp = -2*D0*Dp/(D0*dxp + Dp*dx0)
            A[i,i+1] = (Dxp+Dcor[i+1])/dx0

        A[i,i] = SigA[matid[i]] - ( (Dxm+Dcor[i] + Dxp-Dcor[i+1])/dx0 )

    A = A.tocsr()
    return A


@njit(cache=True)
def diffusion1DX(D, Dcor, SigA, s, dx, matid, albedo):
    NX  = len(s)
    L   = np.zeros(NX)
    U   = np.zeros(NX)
    Axp = np.zeros(NX)

    for i in range(NX):
        D0  = D[i]
        dx0 = dx[i]
        if i==0:
            a = albedo[0]
            Dxm = -(1-a)*D0/(2*(1+a)*D0 + (1-a)*dx0/2)
        else:
            Dm  = D[i-1]
            dxm = dx[i-1]
            Dxm = -2*Dm*D0/(Dm*dx0 + D0*dxm)
            Axm = (Dxm-Dcor[i])/dx0
        if i==(NX-1):
            a = albedo[1]
            Dxp = -(1-a)*D0/(2*(1+a)*D0 + (1-a)*dx0/2)
        else:
            Dp  = D[i+1]
            dxp = dx[i+1]
            Dxp = -2*D0*Dp/(D0*dxp + Dp*dx0)
            Axp[i] = (Dxp+Dcor[i+1])/dx0

        A0 = SigA[matid[i]] - ( (Dxm+Dcor[i] + Dxp-Dcor[i+1])/dx0 )

        if i==0:
            L[i] = 0.0
            U[i] = A0
        else:
            L[i] = Axm/U[i-1]
            U[i] = A0 - Axp[i-1]*L[i]

    B = np.zeros(NX)
    BBefore = 0.0
    for i in range(NX):
        B[i] = s[i] - BBefore*L[i]
        BBefore = B[i]

    flux = np.zeros(NX)
    fluxAfter = 0.0
    for i in range(NX-1, -1, -1):
        fluxAfter = (B[i] - Axp[i]*fluxAfter) / U[i]
        flux[i] = fluxAfter

    return flux

In [ ]:
@njit(cache=True)
def calcFluxDiffusion(SigS, SigA, s, dx, matid, albedo):
    NX = len(s)
    SigT = SigA+SigS
    D = 1/(3*SigT)
    Dcor = np.zeros(NX+1)
    flux = diffusion1DX(D[matid], Dcor, SigA, s, dx, matid, albedo)
    return flux

In [ ]:
def ordinal(i):
    j = i%100
    return "{}".format(i)+({1:"st", 2:"nd", 3:"rd"}.get(j if 14>j>10 else j % 10) or "th")

# **中性子拡散計算による演習**

## **中性子拡散計算プログラムの実行練習**

In [ ]:
LengthX = 12.0 # 平板厚さ
NX = 240 # 空間メッシュ分割数

dx = np.full( NX, LengthX/NX ) # 空間メッシュ幅の設定
xS = np.linspace(start=-LengthX/2, stop=LengthX/2, num=NX+1)
x = (xS[:-1] + xS[1:])/2 # 各空間メッシュの中点位置x座標

matid = np.zeros(NX, dtype=np.int64) # 各メッシュの物質ID指定。左の場合は、全て0

SigA = np.array([0.1]) # 巨視的吸収断面積
SigS = np.array([0.2]) # 巨視的散乱断面積

# s = np.ones(NX)  # 中性子源: すべての空間メッシュで値が「1」
# s = np.full(NX, 1.0) # 中性子源: すべての空間メッシュで値が「1.0」
# s = np.random.rand(NX) # 中性子源: 各空間メッシュで一様乱数に基づいて設定
s = -(x-LengthX/2)*(x+LengthX/2)# 中性子源: 2次関数

albedo = np.array([0.0, 0.0]) # アルベド値：左側境界面、右側境界面
#albedo = np.array([1.0, 1.0]) # アルベド値

flux = calcFluxDiffusion(SigS, SigA, s, dx, matid, albedo)

plt.rcParams["figure.figsize"]=(4,4)
plt.rcParams["font.size"] = 12
plt.plot(x, flux, color="blue", linestyle="solid", label="$\phi(x)$")
plt.plot(x, s, color="red", linestyle="dashed", label="$S(x)$")
plt.title("flux")
plt.xlabel("$x$ (cm)")
plt.ylabel("$\phi(x)$ (a.u.)")
plt.legend()
plt.tight_layout()
plt.show()

## **snapshotデータ行列の準備例**

In [ ]:
np.random.seed(20240619) # 初期乱数の設定

LengthX = 12.0 # 平板厚さ
NX = 240 # 空間メッシュ分割数

dx = np.full( NX, LengthX/NX ) # 空間メッシュ幅の設定
xS = np.linspace(start=-LengthX/2, stop=LengthX/2, num=NX+1)
x = (xS[:-1] + xS[1:])/2 # 各空間メッシュの中点位置x座標

matid = np.zeros(NX, dtype=np.int64) # 各メッシュの物質ID指定。左の場合は、全て0

NC = 400 # 条件数
snapshotF = np.zeros( (NX,NC) ) # NX行, NC列のsnapshot行列を準備
## snapshotQ = np.zeros( (NX,NC) )

plt.rcParams["figure.figsize"]=(4,4)
plt.rcParams["font.size"] = 12

for i in range(NC):
    SigA = np.random.uniform(low=0.01, high=1.0, size=(1)) # 巨視的吸収断面積
    SigS = np.random.uniform(low=0.01, high=1.0, size=(1)) # 巨視的散乱断面積

    # 中性子源の設定
    s = np.zeros(NX)
    xi = np.random.uniform(low=-1.0, high=1.0)
    y = np.random.uniform(low=0.0, high=1.0, size=3)
    a = y[0]/(2*(1+xi)) +y[1]/(xi*xi-1) + y[2]/(2*(1-xi))
    b = (y[2]-y[0])/2
    c = xi*( y[0]/(2*(1+xi)) -y[2]/(2*(1-xi)) )  -y[1]/(xi*xi-1)
    vx = -b/(2*a)
    if vx>=-1 and vx<=1:
        vy = a*vx**2 + b*vx + c
        if vy <0:
            c = c-vy
            vy = vy = a*vx**2 + b*vx + c
            c = c-vy
    stotal = ((a/3)+c)*LengthX
    a = a/stotal
    b = b/stotal
    c = c/stotal
    s = a*(x/(LengthX/2))**2 + b*(x/(LengthX/2)) + c # 中性子源
    if s.min() < 0:
        print("Warning: source is negative")

    albedo = np.random.uniform(low=0.0, high=1.0, size=(2)) # アルベド値

    flux = calcFluxDiffusion(SigS, SigA, s, dx, matid, albedo)

    # snapshotdata
    snapshotF[:,i] = flux
    ## snapshotQ[:,i] = (s + SigS[matid]*flux)

    if i>=10:
        continue
    plt.plot(x, flux, color="blue", linestyle="solid", label="$\phi(x)$")
    plt.plot(x, s, color="red", linestyle="dashed", label="$S(x)$")
    plt.title("flux at {} condition ".format(ordinal(i+1)))
    fluxmax = flux.max()
    plt.xlabel("$x$ (cm)")
    plt.ylabel("$\phi(x)$ (a.u.)")
    plt.legend()
    plt.tight_layout()
    plt.show()

## **snapshotデータ行列の可視化**

In [ ]:
plt.rcParams["figure.figsize"]=(8,8)
plt.rcParams["font.size"] = 12

plt.imshow(snapshotF)
#plt.imshow(snapshotF, cmap="jet")
#plt.imshow(snapshotF, cmap="seismic")
plt.colorbar()
#plt.tight_layout()
plt.show()

## **行列ランクの調べ方**



In [ ]:
rankF = ???????
print("rankF = {}".format(rankF))


## **特異値分解**：特異値分布の可視化

In [ ]:
UF, sF2, VFh = np.linalg.svd(snapshotF, full_matrices=False)

plt.rcParams["figure.figsize"]=(6,4)
plt.scatter(np.arange(len(sF2))+1, sF2, marker=".", color="blue", zorder=2)
plt.vlines(rankF, sF2.min(), sF2.max(), linestyle="dashed", color="silver", zorder=1)
plt.xlabel("POD order")
plt.ylabel("singular value $\sigma_i$")
plt.yscale("log")
plt.tight_layout()
plt.show()

## **POD基底の評価と可視化**

In [ ]:
UF, sF2, VFh = np.linalg.svd(snapshotF, full_matrices=False)

coefF = np.diag(sF2).dot(VFh) # snapshotデータの係数行列
rankF = ????

"""
UF = UF[:,:rankF]
coefF = coefF[:rankF,:]
for i in range(rankF):
    Umin = UF[:,i].min()
    Umax = UF[:,i].max()
    if Umax < abs(Umin):
        UF[:,i]*=-1
        coefF[i,:]*=-1
"""

plt.imshow(UF, cmap="seismic")
plt.show()

plt.imshow(UF[:,:rankF], cmap="seismic") # POD基底行列の列(横方向)について、ランクrまでスライシングして削減
plt.show()


plt.rcParams["figure.figsize"]=(4,4)
for i in range(rankF):
    plt.plot(x, UF[:,i], linestyle="solid", color="black", zorder=2)
    plt.hlines(0, -LengthX/2, LengthX/2, linestyle="dashed", color="silver", zorder=1)
    plt.title("{} order POD".format(ordinal(i+1)))
    plt.xlabel("$x$ (cm)")
    plt.ylabel("$u_{i}(x)$ (a.u.)")
    plt.tight_layout()
    plt.show()


## **データ拡張 (data augumentation)**

In [ ]:
plt.rcParams["figure.figsize"]=(6,4)
plt.imshow(snapshotF)
plt.show()

plt.imshow(snapshotF[::-1, :]) # 上下に反転。すなわちsnapshotデータについて、x方向の境界条件を逆にしたデータとなる
plt.show()

In [ ]:
snapshotDA = np.append(snapshotF, ??????, axis=1)
UF, sF2, VFh = np.linalg.svd(snapshotDA, full_matrices=False)
rankF = np.linalg.matrix_rank(snapshotDA)

print("rankF = {}".format(rankF))
coefF = np.diag(sF2).dot(VFh) # snapshotデータの係数行列

UF = UF[:,:rankF]
coefF = coefF[:rankF,:]
for i in range(rankF):
    Umin = UF[:,i].min()
    Umax = UF[:,i].max()
    if Umax < abs(Umin):
        UF[:,i]*=-1
        coefF[i,:]*=-1

plt.rcParams["figure.figsize"]=(6,4)
plt.scatter(np.arange(len(sF2))+1, sF2, marker=".", color="blue", zorder=2)
plt.vlines(rankF, sF2.min(), sF2.max(), linestyle="dashed", color="silver", zorder=1)
plt.xlabel("POD order")
plt.ylabel("singular value $\sigma_i$")
plt.yscale("log")
plt.tight_layout()
plt.show()

plt.rcParams["figure.figsize"]=(4,4)
for i in range(rankF):
    plt.plot(x, UF[:,i], linestyle="solid", color="black", zorder=2)
    plt.hlines(0, -LengthX/2, LengthX/2, linestyle="dashed", color="silver", zorder=1)
    plt.title("{} order POD".format(ordinal(i+1)))
    plt.xlabel("$x$ (cm)")
    plt.ylabel("$u_{i}(x)$ (a.u.)")
    plt.tight_layout()
    plt.show()

## **低ランク近似**：異なる計算条件におけるPOD基底展開の誤差チェック

In [ ]:
np.random.seed(20240924)

r = 3  # <- 低ランク近似：POD基底の数を変えてみよう
Ncheck = 10 # <- 検証計算する回数。まずは自由に変えてみよう
# Ncheck = 59 # <- 1次のWilks法による(p=95%,q=95%)片側許容限界

LengthX = 12.0 # 平板厚さ
NX = 240 # 空間メッシュ分割数
dx = np.full( NX, LengthX/NX ) # 空間メッシュ幅の設定
xS = np.linspace(start=-LengthX/2, stop=LengthX/2, num=NX+1)
x = (xS[:-1] + xS[1:])/2 # 各空間メッシュの中点位置x座標
matid = np.zeros(NX, dtype=np.int64) # 各メッシュの物質ID指定。左の場合は、全て0


plt.rcParams["figure.figsize"]=(8*r/NX, 8*NX/NX)
plt.imshow(UF[:,:r], cmap="seismic")
#plt.tight_layout()
plt.show()

rRMSEcheck = np.zeros(Ncheck)
MREcheck = np.zeros(Ncheck)

for j in range(Ncheck):
    SigA = np.random.uniform(low=0.01, high=1.0, size=(1)) # 巨視的吸収断面積
    SigS = np.random.uniform(low=0.01, high=1.0, size=(1)) # 巨視的散乱断面積

    # 中性子源の設定
    s = np.zeros(NX)
    xi = np.random.uniform(low=-1.0, high=1.0)
    y = np.random.uniform(low=0.0, high=1.0, size=3)
    a = y[0]/(2*(1+xi)) +y[1]/(xi*xi-1) + y[2]/(2*(1-xi))
    b = (y[2]-y[0])/2
    c = xi*( y[0]/(2*(1+xi)) -y[2]/(2*(1-xi)) )  -y[1]/(xi*xi-1)
    vx = -b/(2*a)
    if vx>=-1 and vx<=1:
        vy = a*vx**2 + b*vx + c
        if vy <0:
            c = c-vy
            vy = vy = a*vx**2 + b*vx + c
            c = c-vy
    stotal = ((a/3)+c)*LengthX
    a = a/stotal
    b = b/stotal
    c = c/stotal
    s = a*(x/(LengthX/2))**2 + b*(x/(LengthX/2)) + c # 中性子源
    if s.min() < 0:
        print("Warning: source is negative")

    albedo = np.random.uniform(low=0.0, high=1.0, size=(2)) # アルベド値

    fluxref = calcFluxDiffusion(SigS, SigA, s, dx, matid, albedo) # 別計算条件で得られた中性子束参照解

    coefPOD = (UF[:,:r].T).dot(fluxref) # 参照解結果に基づいてPOD展開係数を求める
    fluxPOD = UF[:,:r].dot(coefPOD)     # r次までのPOD基底により中性子束を再構成する
    rerr = (fluxPOD/fluxref-1)*100      # 相対誤差 (%)

    rRMSE = np.sqrt( (rerr**2).mean() )
    MRE = (np.abs(rerr)*flux*dx).sum() / ((flux*dx).sum())
    rRMSEcheck[j] = rRMSE
    MREcheck[j] = MRE

    if j < 10:
        plt.rcParams["figure.figsize"]=(4, 4)
        plt.plot(fluxPOD, color="blue", linestyle="dashed", zorder=2)
        plt.plot(fluxref, color="red", linestyle="solid", zorder=1)
        plt.title("{} trial".format(ordinal(j+1)))
        plt.xlabel("$x$ (cm)")
        plt.ylabel("$\phi(x)$ (a.u.)")
        plt.tight_layout()
        plt.show()

        plt.rcParams["figure.figsize"]=(4, 4)
        plt.plot(rerr, color="black", linestyle="solid", zorder=2)
        plt.xlabel("$x$ (cm)")
        plt.title("{} trial".format(ordinal(j+1)))
        plt.ylabel("relative error (%)")
        plt.tight_layout()
        plt.show()

        print("rRMSE = {}, MRE = {}, rerr min = {}, rerr max = {}".format(rRMSE, MRE, rerr.min(), rerr.max()))
        print()


print("rRMSE max = {}".format(rRMSEcheck.max()))
print("MRE max = {}".format(MREcheck.max()))

### **Wilks法とは？**

確率$p$で信頼度$q$を満足する片側許容限界を、ノンパラメトリックな方法で推定することができる手法。

例えば、ランダムサンプリング法により、条件を変えた計算を$N$回だけ計算を実施し、ある量の計算結果を$N$個だけ得ることができたとする。
このとき、降順(あるいは昇順)に並び替えて、上から1番目の値を選ぶだけで、確率$p$で信頼度$q$を満足する片側許容限界を推定することができる。

\\

ここで、サンプリングすべき数(サンプルサイズ)$N$は、1次のWilks法の場合(上から1番目の値を選ぶ場合、要は最大値を調べる場合)には、$N$は以下の式で求めることができる。
$$ N = \left \lceil \frac{\ln(1 - q)}{\ln(p)} \right \rceil  $$

\\
## 参考文献
[N. W. Porter, "Wilks' Formula Applied to Computational Tools: A Practical Discussion and Verification," SAND2019-1901J (2019).](https://www.osti.gov/servlets/purl/1529145)

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
from numba import njit

@njit(cache=True)
def samplingDistribution(size, case=None):
    if case is None or case=="sec3":
        x = np.random.normal(loc=np.pi, scale=np.pi/3, size=size)
        # x = np.random.uniform(low=0.0, high=2*np.pi, size=size)
        # x = np.random.beta(0.8, 1.5, size=size)
    return x


def sampleSizefor1stWilks(p, q):
    return math.ceil(math.log(1-q)/math.log(p))


p = 0.95  # probability of percentile point
q = 0.95  # confidence level
order=1   # order of Wilks formula

Nwilks = sampleSizefor1stWilks(p, q)
print("N = {0} when p={1}, q={2}".format(Nwilks, p, q))


M = 10000 # Wilks法による許容限界判定に成功したかどうか、検証する回数
Np = 10000 # 許容限界値を未満の数を調べる際のサンプリング数

success = np.zeros(M)
tlpq = np.zeros(M)
t=0.0
for i in range(M):
    x = samplingDistribution(Nwilks)
    index = np.argsort(x)[::-1]
    tlpq[i] = x[index[order-1]]

    xtest = samplingDistribution(Np)
    if (xtest < tlpq[i]).sum()/Np >= p:
       success[i] = 1.0

plt.rcParams["figure.figsize"]=(6, 6)
plt.hist(tlpq, bins=25)
plt.title("toleranence limit = {0} +/- {1}".format(tlpq.mean(), tlpq.std(ddof=1)))
plt.xlabel("{0:.1f}/{1:.1f} tolerane limit".format(p*100, q*100))
plt.show()

# calculation of confidence level using bootstrap method
NBS=10000 # bootstrap法によるリサンプリング数
qBS =  np.zeros(NBS)
for b in range(NBS):
   successBS = np.random.choice(success, size=M)
   qBS[b] = successBS.sum()/M

plt.hist(qBS, bins=25)
plt.title("{0} +/- {1}".format(qBS.mean(), qBS.std(ddof=1)))
plt.xlabel("q_calc")
plt.show()

## **POD拡散計算の例**

In [ ]:
np.random.seed(20240925)

r = 3  # <- 低ランク近似：POD基底の数を変えてみよう
Ncheck = 1 # <- 検証計算する回数。まずは自由に変えてみよう
# Ncheck = 59 # <- 1次のWilks法による(p=95%,q=95%)片側許容限界

LengthX = 12.0 # 平板厚さ
NX = 240 # 空間メッシュ分割数
dx = np.full( NX, LengthX/NX ) # 空間メッシュ幅の設定
xS = np.linspace(start=-LengthX/2, stop=LengthX/2, num=NX+1)
x = (xS[:-1] + xS[1:])/2 # 各空間メッシュの中点位置x座標
matid = np.zeros(NX, dtype=np.int64) # 各メッシュの物質ID指定。左の場合は、全て0

plt.rcParams["figure.figsize"]=(8*r/NX, 8*NX/NX)
plt.imshow(UF[:,:r], cmap="seismic")
#plt.tight_layout()
plt.show()

rRMSEcheck = np.zeros(Ncheck)
MREcheck = np.zeros(Ncheck)

for j in range(Ncheck):
    SigA = np.random.uniform(low=0.01, high=1.0, size=(1)) # 巨視的吸収断面積
    SigS = np.random.uniform(low=0.01, high=1.0, size=(1)) # 巨視的散乱断面積
    SigT = SigA + SigS                                     # 巨視的全断面積
    D = 1.0/(3.0*SigT)                                     # 拡散係数

    # 中性子源の設定
    s = np.zeros(NX)
    xi = np.random.uniform(low=-1.0, high=1.0)
    y = np.random.uniform(low=0.0, high=1.0, size=3)
    a = y[0]/(2*(1+xi)) +y[1]/(xi*xi-1) + y[2]/(2*(1-xi))
    b = (y[2]-y[0])/2
    c = xi*( y[0]/(2*(1+xi)) -y[2]/(2*(1-xi)) )  -y[1]/(xi*xi-1)
    vx = -b/(2*a)
    if vx>=-1 and vx<=1:
        vy = a*vx**2 + b*vx + c
        if vy <0:
            c = c-vy
            vy = vy = a*vx**2 + b*vx + c
            c = c-vy
    stotal = ((a/3)+c)*LengthX
    a = a/stotal
    b = b/stotal
    c = c/stotal
    s = a*(x/(LengthX/2))**2 + b*(x/(LengthX/2)) + c # 中性子源
    if s.min() < 0:
        print("Warning: source is negative")

    albedo = np.random.uniform(low=0.0, high=1.0, size=(2)) # アルベド値

    fluxref = calcFluxDiffusion(SigS, SigA, s, dx, matid, albedo) # 別計算条件で得られた中性子束参照解

    A = calcMatrixA(1/(3*SigT[matid]), np.zeros(NX+1), SigA, dx, matid, albedo) # 中性子拡散計算で用いる係数行列Aの計算

    Atilde = (UF[:,:r].T).dot(A.dot(UF[:,:r]))  # サンドイッチ法による係数行列Aの圧縮
    stilde = (UF[:,:r].T).dot(s)                # 中性子源sのPOD展開係数評価
    coefPOD = np.linalg.solve(Atilde, stilde)   # 次元圧縮した拡散計算連立方程式を解く

    fluxPOD = UF[:,:r].dot(coefPOD)     # r次までのPOD基底により中性子束を再構成する
    rerr = (fluxPOD/fluxref-1)*100      # 相対誤差 (%)

    rRMSE = np.sqrt( (rerr**2).mean() )
    MRE = (np.abs(rerr)*flux*dx).sum() / ((flux*dx).sum())
    rRMSEcheck[j] = rRMSE
    MREcheck[j] = MRE

    if j < 10:
        plt.rcParams["figure.figsize"]=(4,4)
        plt.scatter(np.arange(r)+1,  coefPOD, marker=".", color="blue")
        plt.xlabel("POD order")
        plt.ylabel("POD expansion coefficient (a.u.)")
        plt.tight_layout()
        plt.tight_layout()
        plt.show()

        plt.rcParams["figure.figsize"]=(4, 4)
        plt.plot(fluxPOD, color="blue", linestyle="dashed", zorder=2)
        plt.plot(fluxref, color="red", linestyle="solid", zorder=1)
        plt.title("{} trial".format(ordinal(j+1)))
        plt.xlabel("$x$ (cm)")
        plt.ylabel("$\phi(x)$ (a.u.)")
        plt.tight_layout()
        plt.show()

        plt.rcParams["figure.figsize"]=(4, 4)
        plt.plot(rerr, color="black", linestyle="solid", zorder=2)
        plt.xlabel("$x$ (cm)")
        plt.title("{} trial".format(ordinal(j+1)))
        plt.ylabel("relative error (%)")
        plt.tight_layout()
        plt.show()

        print("rRMSE = {}, MRE = {}, rerr min = {}, rerr max = {}".format(rRMSE, MRE, rerr.min(), rerr.max()))
        print()


print("rRMSE max = {}".format(rRMSEcheck.max()))
print("MRE max = {}".format(MREcheck.max()))